## 1. Set-up

In [1]:
# Mount Drive files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
MODEL_VERSION = "multimodal"

In [3]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 95.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 55.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 19.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 12.9 MB/s eta 0:00:0

In [4]:
import numpy as np
import pandas as pd

import transformers
from datasets import Dataset,load_dataset,load_from_disk, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertForSequenceClassification, BertTokenizer, Trainer, VisualBertModel
from scipy.stats import pearsonr
import sklearn.metrics
from datasets import load_metric, load_dataset
from sklearn.metrics import mean_squared_error

## 2. Arrange datasets and tokenize them

In this section, I'll use the dataset provided from

In [5]:

#dataset = load_dataset("PlanTL-GOB-ES/sts-es")
dataset = load_dataset("nflechas/semantic_sentence_similarity_ES")
dataset

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1411 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/245 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/168 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/nflechas___parquet/nflechas--semantic_sentence_similarity_ES-46a45b872408dfc2/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label'],
        num_rows: 1411
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label'],
        num_rows: 245
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label'],
        num_rows: 168
    })
})

In [6]:
#dataset["test"]["sentence1"][:5]

Tokenize datasets

In [7]:
#model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=1)
model = BertForSequenceClassification.from_pretrained("uclanlp/visualbert-vqa-coco-pre", num_labels=1)
#model = VisualBertModel.from_pretrained("uclanlp/visualbert-vqa-coco-pre", num_labels=1)
tokenizer = BertTokenizer.from_pretrained("bert-base-cased", model_max_length=512)

You are using a model of type visual_bert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Some weights of the model checkpoint at uclanlp/visualbert-vqa-coco-pre were not used when initializing BertForSequenceClassification: ['visual_bert.encoder.layer.1.attention.self.query.bias', 'visual_bert.encoder.layer.8.output.dense.weight', 'visual_bert.encoder.layer.4.output.LayerNorm.bias', 'visual_bert.encoder.layer.1.attention.self.query.weight', 'visual_bert.encoder.layer.11.attention.self.value.weight', 'visual_bert.encoder.layer.10.attention.output.LayerNorm.bias', 'visual_bert.pooler.dense.weight', 'visual_bert.encoder.layer.7.output.LayerNorm.bias', 'visual_bert.encoder.layer.0.intermediate.dense.weight', 'visual_bert.encoder.layer.6.attention.self.query.bias', 'visual_bert.encoder.layer.9.attention.output.dense.bias', 'visual_bert.encoder.layer.11.output.LayerNorm.bias', 'visual_bert.encoder.layer.0.output.dense.weight', 'visual_bert.encoder.layer.2.attention.self.key.weight', 'cls.predictions.bias', 'visual_bert.encoder.layer.10.attention.self.key.bias', 'visual_bert.enco

In [ ]:
# examples = [("monitos en la cama", "estamos en la cama"), ("tres tristes tigres", "los tigres son lindos")]

# tokenizer(examples, padding="max_length", truncation=True)

In [ ]:
# print(dataset['train']['sentence1'][20])
# print(dataset['train']['sentence2'][20])

# a = tokenizer(dataset['train']['sentence1'][20], dataset['train']['sentence2'][20], padding=True, truncation=True)
# tokenizer.decode(a["input_ids"])

In [8]:
def tokenize_function(examples):
    return tokenizer(examples["sentence1"], examples["sentence2"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/1411 [00:00<?, ? examples/s]

Map:   0%|          | 0/245 [00:00<?, ? examples/s]

Map:   0%|          | 0/168 [00:00<?, ? examples/s]

In [ ]:
#tokenized_datasets

In [ ]:
for input_idd in tokenized_datasets["train"]["input_ids"]:
  if len(input_idd) != 512:
    print(len(input_idd))

We use rmse as a metric since we are in a regression scenario
 (the lower this value, the better the model performs)

In [13]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    rmse = mean_squared_error(labels, predictions, squared=False)
    return {"rmse": rmse}

## 3. Train the model

In [14]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir=f"/content/drive/MyDrive/Colab Notebooks/dl4nlp_labs/final_project/sentence_similarity/models/{MODEL_VERSION}",
                                  logging_strategy="epoch",
                                  evaluation_strategy="epoch",
                                  #evaluation_strategy="steps",
                                  eval_steps = 10,
                                  per_device_train_batch_size=16,
                                  per_device_eval_batch_size=16,
                                  num_train_epochs = 5,
                                  save_total_limit = 2,
                                  #save_strategy = 'steps',
                                  save_strategy = 'epoch',
                                  #load_best_model_at_end=False
                                  load_best_model_at_end=True)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics
)

trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rmse
1,1.272700,1.392211,1.179920
2,1.065000,1.431930,1.196633
3,1.074000,1.411559,1.188091
4,0.930100,1.468485,1.211811
5,0.737100,1.500781,1.225063


TrainOutput(global_step=445, training_loss=1.0157714329408796, metrics={'train_runtime': 727.4165, 'train_samples_per_second': 9.699, 'train_steps_per_second': 0.612, 'total_flos': 1856231829058560.0, 'train_loss': 1.0157714329408796, 'epoch': 5.0})

## 4. Save model

In [ ]:
#model.save_pretrained(f"/content/drive/MyDrive/Colab Notebooks/MT-model/models_tokenizers/model_{MODEL_VERSION}/checkpoint-249")
#tokenizer.save_pretrained(f"/content/drive/MyDrive/Colab Notebooks/MT-model/models_tokenizers/tokenizer_{MODEL_VERSION}/checkpoint-249")

NameError: ignored

In [22]:

# # load the model/tokenizer
model = BertForSequenceClassification.from_pretrained(f"/content/drive/MyDrive/Colab Notebooks/dl4nlp_labs/final_project/sentence_similarity/models/{MODEL_VERSION}/checkpoint-445")
tokenizer = BertTokenizer.from_pretrained("bert-base-cased", model_max_length=512)


## 5. New predictions

In [15]:
tokenized_test_dataset = tokenized_datasets["test"]

In [16]:
trainer = Trainer(model=model)

# def tokenize_function(examples):
#     return tokenizer(examples["text"], padding="max_length", truncation=True)

def prediction(tokenized_datasets):
    # df=pd.DataFrame({'text':texts})
    # dataset = Dataset.from_pandas(df,preserve_index=False) 
    # tokenized_datasets = dataset.map(tokenize_function)
    raw_pred, _, _ = trainer.predict(tokenized_datasets) 
    return(raw_pred)

def evaluate(preds, gold):
    # Compute pearson:
    pearson = pearsonr(preds, gold)[0]
      
    # Compute rmse
    rmse = mean_squared_error(preds, gold, squared=False)

    print(f"RMSE = {rmse:.6f}")
    print(f"Pearson: {pearson:.6f}")

In [23]:
final_preds = prediction(tokenized_test_dataset)
pred_list = [pred[0] for pred in final_preds]
gold = tokenized_test_dataset['label']
evaluate(pred_list, gold)

RMSE = 1.179920
Pearson: 0.194883


In [ ]:
final_preds = prediction(tokenized_test_dataset)
pred_list = [pred[0] for pred in final_preds]
gold = tokenized_test_dataset['label']
evaluate(pred_list, gold)

RMSE = 0.899621
Pearson: 0.369880
